# Common Libraries

In [2]:
import gzip
import re
import xml.etree.ElementTree as ET
import pandas as pd

# Params

In [3]:
brodman_nifti_path = "/usr/local/afni/abin/MNI_Glasser_HCP_v1.0.nii.gz"
save_xml_data_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/metadata.xml"

is_save_xml = True
save_info_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/MNI_Glasser_HCP_v1.0_info.csv"

# Load data

In [4]:
with gzip.open(brodman_nifti_path, 'rt', errors='ignore') as f:
    content = f.read()

# Filtering meta data

In [5]:
metadata = re.findall(r'<AFNI_attributes.*?</AFNI_attributes>', content, re.DOTALL)[0]

if is_save_xml:
    with open(save_xml_data_path, 'w') as xml_file:
        xml_file.write(metadata)

# Read meta data

In [6]:
root = ET.fromstring(metadata)
value_label_dtable = None
for atr in root.findall(".//AFNI_atr"):
    if atr.get('atr_name') == "ATLAS_LABEL_TABLE":
        value_label_dtable = atr.text

In [7]:
pattern_corrected = r'<ATLAS_POINT.*?/>'
matches_corrected = re.findall(pattern_corrected, value_label_dtable, re.DOTALL)
matches_corrected = [e.replace("\n", "") for e in matches_corrected]

In [19]:
pattern_struct_val = r'STRUCT="([^"]+)"  VAL="([^"]+)"'

In [27]:
for i, row in enumerate(matches_corrected):
    try:
        pattern_matched = re.findall(pattern_struct_val, row)[0]
    except:
        print(i, row)

In [26]:
matches_corrected[62] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Area_23d"  VAL="032"  OKEY="032"  GYoAR="0"  COG = "3.133330 17.500000 39.000000"  SB_LABEL="L_Area_23d" />'
matches_corrected[79] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="R_Dorsal_Area_24d"  VAL="1040"  OKEY="1040"  GYoAR="0"  COG = "-8.333340 16.222200 51.777800"  SB_LABEL="R_Dorsal_Area_24d" />'
matches_corrected[94] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="R_Area_Lateral_IntraParietal_ventral"  VAL="1048"  OKEY="1048"  GYoAR="0"  COG = "-24.769200 56.307700 53.230800"  SB_LABEL="R_Area_Lateral_IntraParietal_ventral" />'
matches_corrected[99] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Area_1"  VAL="051"  OKEY="051"  GYoAR="0"  COG = "51.476200 23.952400 59.142900"  SB_LABEL="L_Area_1" />'
matches_corrected[105] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Dorsal_area_6"  VAL="054"  OKEY="054"  GYoAR="0"  COG = "36.074100 15.666700 67.518500"  SB_LABEL="L_Dorsal_area_6" />'
matches_corrected[144] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Area_44"  VAL="074"  OKEY="074"  GYoAR="0"  COG = "56.533300 -16.600000 12.500000"  SB_LABEL="L_Area_44" />'
matches_corrected[176] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Area_11l"  VAL="091"  OKEY="091"  GYoAR="0"  COG = "28.000000 -47.083300 -13.000000"  SB_LABEL="L_Area_11l" />'
matches_corrected[214] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Piriform_Cortex"  VAL="110"  OKEY="110"  GYoAR="0"  COG = "32.000000 -7.250000 -16.500000"  SB_LABEL="L_Piriform_Cortex" />'
matches_corrected[219] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="R_Anterior_Agranular_Insula_Complex"  VAL="1112"  OKEY="1112"  GYoAR="0"  COG = "-30.871000 -17.774200 -13.129000"  SB_LABEL="R_Anterior_Agranular_Insula_Complex" />'
matches_corrected[287] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="R_Area_IntraParietal_0"  VAL="1146"  OKEY="1146"  GYoAR="0"  COG = "-34.166700 75.000000 27.233300"  SB_LABEL="R_Area_IntraParietal_0" />'
matches_corrected[336] = '<ATLAS_POINT  data_type="atlas_point"   STRUCT="L_Area_posterior_47r"  VAL="171"  OKEY="171"  GYoAR="0"  COG = "41.500000 -42.384600 -2.384610"  SB_LABEL="L_Area_posterior_47r" />'

In [28]:
meta_data_mapping = [re.findall(pattern_struct_val, row)[0] for row in matches_corrected]

In [29]:
meta_data_info = {}
for description, value in meta_data_mapping:
    meta_data_info[int(value)] = description

# Save data

In [31]:
save_info = pd.DataFrame({ "label" : list(meta_data_info.keys()), "description" : list(meta_data_info.values()) })
save_info.to_csv(save_info_path, index=False)

In [35]:
save_info_path

'/home/seojin/Seojin_commonTool/Module/Brain_ROI/MNI_Glasser_HCP_v1.0_info.csv'